# Azure Backend API Testing

This notebook allows direct testing of the Azure backend API endpoints.

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import time

In [2]:
# Configuration
BASE_URL = "https://your-azure-backend-url.com"  # Replace with your actual Azure URL
API_KEY = "your-api-key"  # Replace with your actual API key

# Headers - API key goes in X-Stainless-API-Key header for this backend
headers = {
    "Content-Type": "application/json",
    "X-Stainless-API-Key": API_KEY if API_KEY else None,
}

# Remove None values from headers
headers = {k: v for k, v in headers.items() if v is not None}

## Test Functions

In [3]:
def test_endpoint(endpoint, method="GET", data=None, params=None):
    """
    Test an API endpoint and return the response
    """
    url = f"{BASE_URL.rstrip('/')}/{endpoint.lstrip('/')}"

    print(f"Testing: {method} {url}")
    print(f"Headers: {headers}")

    if data:
        print(f"Data: {json.dumps(data, indent=2)}")

    start_time = time.time()

    try:
        if method.upper() == "GET":
            response = requests.get(url, headers=headers, params=params)
        elif method.upper() == "POST":
            response = requests.post(url, headers=headers, json=data, params=params)
        elif method.upper() == "PUT":
            response = requests.put(url, headers=headers, json=data, params=params)
        elif method.upper() == "DELETE":
            response = requests.delete(url, headers=headers, params=params)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")

        response_time = time.time() - start_time

        print(f"\nResponse Status: {response.status_code}")
        print(f"Response Time: {response_time:.2f}s")
        print(f"Response Headers: {dict(response.headers)}")

        try:
            response_json = response.json()
            print(f"Response Body: {json.dumps(response_json, indent=2)}")
        except json.JSONDecodeError:
            print(f"Response Body (text): {response.text}")

        return {
            "status_code": response.status_code,
            "response_time": response_time,
            "response": response,
            "timestamp": datetime.now().isoformat(),
        }

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return {"error": str(e), "timestamp": datetime.now().isoformat()}

## Health Check

In [4]:
# Test health endpoint
health_result = test_endpoint("/health")
health_result

Testing: GET https://your-azure-backend-url.com/health
Headers: {'Content-Type': 'application/json', 'X-Stainless-API-Key': 'your-api-key'}
Request failed: HTTPSConnectionPool(host='your-azure-backend-url.com', port=443): Max retries exceeded with url: /health (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x116e4e990>: Failed to resolve 'your-azure-backend-url.com' ([Errno 8] nodename nor servname provided, or not known)"))


{'error': 'HTTPSConnectionPool(host=\'your-azure-backend-url.com\', port=443): Max retries exceeded with url: /health (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x116e4e990>: Failed to resolve \'your-azure-backend-url.com\' ([Errno 8] nodename nor servname provided, or not known)"))',
 'timestamp': '2025-07-07T23:01:09.067098'}

## API Endpoints Testing

In [5]:
# Chat Completions API - Main LLM endpoint
chat_data = {
    "messages": [{"role": "user", "content": "Hello! Can you help me test this API?"}],
    "model": "gpt-4",
    "temperature": 0.7,
    "max_tokens": 100,
    "provider_constraint": ["openai", "anthropic"],
    "cost_bias": 0.5,
}

chat_result = test_endpoint("/v1/chat/completions", method="POST", data=chat_data)
chat_result

Testing: POST https://your-azure-backend-url.com/v1/chat/completions
Headers: {'Content-Type': 'application/json', 'X-Stainless-API-Key': 'your-api-key'}
Data: {
  "messages": [
    {
      "role": "user",
      "content": "Hello! Can you help me test this API?"
    }
  ],
  "model": "gpt-4",
  "temperature": 0.7,
  "max_tokens": 100,
  "provider_constraint": [
    "openai",
    "anthropic"
  ],
  "cost_bias": 0.5
}
Request failed: HTTPSConnectionPool(host='your-azure-backend-url.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1135b8110>: Failed to resolve 'your-azure-backend-url.com' ([Errno 8] nodename nor servname provided, or not known)"))


{'error': 'HTTPSConnectionPool(host=\'your-azure-backend-url.com\', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1135b8110>: Failed to resolve \'your-azure-backend-url.com\' ([Errno 8] nodename nor servname provided, or not known)"))',
 'timestamp': '2025-07-07T23:01:09.084480'}

In [6]:
# POST request example
post_data = {"key": "value", "timestamp": datetime.now().isoformat()}

post_result = test_endpoint("/api/your-endpoint", method="POST", data=post_data)
post_result

Testing: POST https://your-azure-backend-url.com/api/your-endpoint
Headers: {'Content-Type': 'application/json', 'X-Stainless-API-Key': 'your-api-key'}
Data: {
  "key": "value",
  "timestamp": "2025-07-07T23:18:26.449774"
}
Request failed: HTTPSConnectionPool(host='your-azure-backend-url.com', port=443): Max retries exceeded with url: /api/your-endpoint (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x15e9a1ee0>: Failed to resolve 'your-azure-backend-url.com' ([Errno 8] nodename nor servname provided, or not known)"))


{'error': 'HTTPSConnectionPool(host=\'your-azure-backend-url.com\', port=443): Max retries exceeded with url: /api/your-endpoint (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x15e9a1ee0>: Failed to resolve \'your-azure-backend-url.com\' ([Errno 8] nodename nor servname provided, or not known)"))',
 'timestamp': '2025-07-07T23:18:26.493868'}

## Batch Testing

In [7]:
# Test multiple endpoints
endpoints_to_test = [
    {"name": "Health Check", "endpoint": "/health", "method": "GET"},
    {"name": "API Endpoint 1", "endpoint": "/api/endpoint1", "method": "GET"},
    {
        "name": "API Endpoint 2",
        "endpoint": "/api/endpoint2",
        "method": "POST",
        "data": {"test": "data"},
    },
]

results = []

for test_case in endpoints_to_test:
    print(f"\n{'='*50}")
    print(f"Testing: {test_case['name']}")
    print(f"{'='*50}")

    result = test_endpoint(
        test_case["endpoint"], method=test_case["method"], data=test_case.get("data")
    )

    result["test_name"] = test_case["name"]
    result["endpoint"] = test_case["endpoint"]
    result["method"] = test_case["method"]

    results.append(result)

    print(f"\nResult: {result}")


Testing: Health Check
Testing: GET https://your-azure-backend-url.com/health
Headers: {'Content-Type': 'application/json', 'X-Stainless-API-Key': 'your-api-key'}
Request failed: HTTPSConnectionPool(host='your-azure-backend-url.com', port=443): Max retries exceeded with url: /health (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x15e9a3410>: Failed to resolve 'your-azure-backend-url.com' ([Errno 8] nodename nor servname provided, or not known)"))

Result: {'error': 'HTTPSConnectionPool(host=\'your-azure-backend-url.com\', port=443): Max retries exceeded with url: /health (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x15e9a3410>: Failed to resolve \'your-azure-backend-url.com\' ([Errno 8] nodename nor servname provided, or not known)"))', 'timestamp': '2025-07-07T23:18:32.650703', 'test_name': 'Health Check', 'endpoint': '/health', 'method': 'GET'}

Testing: API Endpoint 1
Testing: GET https://your-azure-backend-url.com/ap

## Results Analysis

In [8]:
# Convert results to DataFrame for analysis
if results:
    df = pd.DataFrame(results)
    print("Test Results Summary:")
    print(
        df[
            [
                "test_name",
                "endpoint",
                "method",
                "status_code",
                "response_time",
                "timestamp",
            ]
        ]
    )

    # Save results to CSV
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"api_test_results_{timestamp}.csv"
    df.to_csv(filename, index=False)
    print(f"\nResults saved to: {filename}")
else:
    print("No test results to analyze")

Test Results Summary:


KeyError: "['status_code', 'response_time'] not in index"

## Custom Testing Cell

In [ ]:
# Use this cell for custom API testing
# Replace with your specific endpoint and data

custom_endpoint = "/api/your-custom-endpoint"
custom_data = {
    # Add your custom data here
}

custom_result = test_endpoint(custom_endpoint, method="POST", data=custom_data)
custom_result